In [2]:
import socket
import struct
import time

IP = "127.0.0.1"
PORT_SERVER = 27015
PORT_CLIENT = 27016

sock_send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv.bind((IP, PORT_CLIENT))

def inofs_send(s: str="", *args):
	b = b""
	if (len(args) > 0):
		b = struct.pack("<" + s[0]*len(args), *args)
	sock_send.sendto(bytes(s, "ascii") + b, (IP, PORT_SERVER))
    
def inofs_recv(format: str):
	data, _ = sock_recv.recvfrom(1024)
	return struct.unpack(format, data)

In [7]:
# Set up throttle control
inofs_send("dC;310A:c;089A:s;")
for i in range(11):
	inputs = 8
	throttle = 16384/10*i
	inofs_send(inputs, throttle)
	time.sleep(0.1);
inofs_send("dC;")

# Set up TAS speed monitor
inofs_send("fM;2B8:i;")
for i in range(10):
	values = inofs_recv("<ff")
	print("TAS: " + str(values[1]/128) + " knots")
inofs_send("fM;")

TypeError: 'int' object is not subscriptable

In [36]:
# Read if the sim is paused
inofs_send("fR:2;264:us;")
data, _ = sock_recv.recvfrom(1024)
print(len(data))
values = struct.unpack("<f", data)
print(str(values[0]))

8


error: unpack requires a buffer of 4 bytes

In [18]:
# Set game pause
pause = True
inofs_send("dW;262:us;", pause)